Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

In [1]:
from scipy.stats import ttest_1samp
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
import numpy as np
import pandas as pd


from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm
from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
df = pd.read_csv('cookie_cats 2.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [3]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [5]:
df.version.value_counts()

version
gate_40    45489
gate_30    44700
Name: count, dtype: int64

In [6]:
df.userid.value_counts().sort_values()

userid
116        1
488        1
540        1
1066       1
1444       1
          ..
9994217    1
9997049    1
9996999    1
9974108    1
9999861    1
Name: count, Length: 90189, dtype: int64

Наши группы близки по количеству пользователей и наши пользователи не повторяются, userid уникальны

Проверим распределение на нормальность в каждой группе

In [7]:
# Контроль нормального распределения для gate_30:
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_30", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.0881, p-value = 0.0000


In [8]:
# Контроль нормального распределения для gate_40:
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.4826, p-value = 0.0000


Поскольку для обеих групп  p-value<0.05, то мы можем сделать следующее заключение:
неверно предположение о нормальном распределении, для обоих групп.

Используем критерий Манна-Уитни для проверки влияния увеличения сохраниения до 40 уровня на количество сыгранных раундов.

In [9]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 1024331250.5000, p-value = 0.0502


p-value > 0.05, значит статистически значимых различий не обнаружено.

Рассмотрим удержание в течение одного дня по группам

In [10]:
df.retention_1.groupby(df.version).sum()/df.retention_1.groupby(df.version).count()


version
gate_30    0.448188
gate_40    0.442283
Name: retention_1, dtype: float64

Мы имеем практически одинаковые значения по группам.

Рассмотрим удержание в течение 7 дней по группам

In [11]:
df.groupby('version')['retention_7'].sum() / df.groupby('version')['retention_7'].count()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Незначительно выше доля в группе 'gate_40'

Вывод: в целом не обнаружено статистически значимых различий при увеличении автосохранения до 'gate_40' в части количества сыгранных раундов. Удержание клиентов в течение одного дня практически одинаковое по группам. Удержание в течение 7 дней несколько выше в группе 'gate_30'.
Следовательно не рекомендуем увеличивать аквтосохранение до уровня 'gate_40'.